In [7]:
import os.path
import cv2
import matplotlib.pyplot as plt
import numpy as np
from glob import glob
from pycocotools import mask
from PIL import Image, ImageDraw
from tqdm import tqdm
import json



In [5]:
def encode_segmentation(segmentation_arr):
    """
    Encode a binary segmentation (np.array) to RLE format using the pycocotools Mask API.
    Args:
        segmentation_arr (np.array): [h x w] binary segmentation
    Returns:
		Rs (dict): the encoded mask in RLE format
    """
    segmentation = np.asfortranarray(segmentation_arr.astype('uint8'))
    Rs = mask.encode(segmentation)
    Rs['counts'] = Rs['counts'].decode()
    return Rs

In [6]:
LOCALIZATION_TASKS = [
    "Enlarged Cardiomediastinum",
    "Cardiomegaly",
    "Lung Lesion",
    "Airspace Opacity",
    "Edema",
    "Consolidation",
    "Atelectasis",
    "Pneumothorax",
    "Pleural Effusion",
    "Support Devices",
]

In [8]:
def create_mask(polygons, img_dims):
    """
    Creates a binary mask (of the original matrix size) given a list of polygon
        annotations format.

    Args:
        polygons (list): [[[x11,y11],[x12,y12],...[x1n,y1n]],...]

    Returns:
        mask (np.array): binary mask, 1 where the pixel is predicted to be the,
                                                 pathology, 0 otherwise
    """
    poly = Image.new("1", (img_dims[1], img_dims[0]))
    for polygon in polygons:
        coords = [(point[0], point[1]) for point in polygon]
        ImageDraw.Draw(poly).polygon(coords, outline=1, fill=1)

    binary_mask = np.array(poly, dtype="int")
    return binary_mask


def ann_to_mask(input_path, output_path):
    """
    Args:
        input_path (string): json file path with raw human annotations
        output_path (string): json file path for saving encoded segmentations
    """
    print(f"Reading annotations from {input_path}...")
    with open(input_path) as f:
        ann = json.load(f)

    print(f"Creating and encoding segmentations...")
    results = {}
    for img_id in tqdm(ann.keys()):
        if img_id not in results:
            results[img_id] = {}
        for task in LOCALIZATION_TASKS:
            # create segmentation
            polygons = ann[img_id][task] if task in ann[img_id] else []
            img_dims = ann[img_id]["img_size"]
            segm_map = create_mask(polygons, img_dims)

            # encode to coco mask
            encoded_map = encode_segmentation(segm_map)
            results[img_id][task] = encoded_map

    assert len(results.keys()) == len(ann.keys())

    # save segmentations to json file
    print(f"Segmentation masks saved to {output_path}")
    with open(output_path, "w") as outfile:
        json.dump(results, outfile)